In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
import matplotlib.pyplot as plt
import pickle

In [2]:
BSC = pd.read_csv('./original_data/Base_station_basic_info.csv')
CLD = pd.read_csv('./original_data/Cell-level_data.csv')
ECD = pd.read_csv('./original_data/Energy_consumption_data.csv')
PCD = pd.read_csv('./original_data/Power_consumption_prediction.csv') #26139

In [3]:
PCD['Time'] =  pd.to_datetime(PCD['Time'], format='%Y-%m-%d %H:%M:%S')

In [4]:
#TAK POWINNO BYC 
merged_df = CLD.merge(BSC, on=['BS', 'CellName'], how='inner')
fullmerged_df = merged_df.merge(ECD, on=['BS', 'Time'], how='left') 

CLD['Time'] =  pd.to_datetime(CLD['Time'], format='%m/%d/%Y %H:%M')
CLD_PCD = CLD.merge(PCD, on=['BS', 'Time'], how='inner')
CLD_PCD_BSC = CLD_PCD.merge(BSC, on=['BS', 'CellName'], how='inner')

In [5]:
CLD_PCD_BSC['Year'] = CLD_PCD_BSC['Time'].dt.year
CLD_PCD_BSC['Day'] = CLD_PCD_BSC['Time'].dt.day
CLD_PCD_BSC['Hour'] = CLD_PCD_BSC['Time'].dt.hour
CLD_PCD_BSC['Month'] = CLD_PCD_BSC['Time'].dt.month

In [6]:
unique_cells_per_base = fullmerged_df.groupby('BS')['CellName'].nunique()
one_cell_bases = unique_cells_per_base[unique_cells_per_base == 1].index
two_cell_bases = unique_cells_per_base[unique_cells_per_base == 2].index
four_cell_bases = unique_cells_per_base[unique_cells_per_base == 4].index

eval_data_1cell = CLD_PCD_BSC[CLD_PCD_BSC['BS'].isin(one_cell_bases)]
eval_data_2cell = CLD_PCD_BSC[CLD_PCD_BSC['BS'].isin(two_cell_bases)]
eval_data_4cell = CLD_PCD_BSC[CLD_PCD_BSC['BS'].isin(four_cell_bases)]
# eval_data_24cell = CLD_PCD_BSC[CLD_PCD_BSC['BS'].isin(four_cell_bases) | CLD_PCD_BSC['BS'].isin(two_cell_bases)]

# print(eval_data_4cell.value_counts('BS'))

ONE CELL PREP

In [7]:
# eval_data_1cell = eval_data_1cell.drop(["CellName", "BS", "Year", "Month", "Time", "w"], axis=1)
eval_data_1cell = eval_data_1cell.drop(["CellName", "Year", "Month", "w"], axis=1)
# eval_data_1cell.to_csv('./a.csv')
print(eval_data_1cell.columns)

with open('./pickle/past_energy_cell1_dict.pkl', 'rb') as file:
    past_energy_cell1_dict = pickle.load(file)
eval_data_1cell['Energy_1'] = np.zeros(len(eval_data_1cell))
for row in eval_data_1cell.itertuples():
    eval_data_1cell.at[row.Index, 'Energy_1'] = past_energy_cell1_dict[(row.BS,row.Time)]

eval_data_1cell.to_csv('./a.csv')
new_column_order = ['Time', 'BS','Bandwidth', 'ESMode1','ESMode2','ESMode3','ESMode4','ESMode5','ESMode6','Frequency', 'TXpower', 'load','Mode','Antennas','Day','Hour','RUType','Energy_1']
eval_data_1cell = eval_data_1cell[new_column_order]
print(eval_data_1cell.columns)

Index(['Time', 'BS', 'load', 'ESMode1', 'ESMode2', 'ESMode3', 'ESMode4',
       'ESMode5', 'ESMode6', 'Energy', 'RUType', 'Mode', 'Frequency',
       'Bandwidth', 'Antennas', 'TXpower', 'Day', 'Hour'],
      dtype='object')
Index(['Time', 'BS', 'Bandwidth', 'ESMode1', 'ESMode2', 'ESMode3', 'ESMode4',
       'ESMode5', 'ESMode6', 'Frequency', 'TXpower', 'load', 'Mode',
       'Antennas', 'Day', 'Hour', 'RUType', 'Energy_1'],
      dtype='object')


In [8]:
label_encoders = {}
object_cols = ['RUType', 'Mode'] 

for col in object_cols:
    le = LabelEncoder()
    eval_data_1cell[col] = le.fit_transform(eval_data_1cell[col])
    label_encoders[col] = le

eval_data_1cell['RUType'].value_counts()

RUType
6     6419
8     5521
9     3446
7     1133
0      913
4      904
5      872
2      767
3      527
11     130
1      101
10      54
Name: count, dtype: int64

In [9]:
Encoded_rutype = torch.nn.functional.one_hot(torch.tensor(eval_data_1cell["RUType"].values).long(), 12)
Encoded_rutype = pd.DataFrame(Encoded_rutype, columns=[f"RUType{i}" for i in range(12)])

In [10]:
# CLD_PCD_BSC_no_dupa = pd.concat([CLD_PCD_BSC_no_dupa, Encoded_cellname], axis=1)
# CLD_PCD_BSC_no_dupa = pd.concat([CLD_PCD_BSC_no_dupa, Encoded_rutype], axis=1)

eval_data_1cell_white_man = eval_data_1cell.to_numpy()
Encoded_rutype_white_man = Encoded_rutype.to_numpy()
chuj = eval_data_1cell.columns
eval_data_1cell = np.concatenate((eval_data_1cell_white_man,Encoded_rutype_white_man), axis=1)
eval_data_1cell = pd.DataFrame(eval_data_1cell, columns = chuj.append(Encoded_rutype.columns))

eval_data_1cell = eval_data_1cell.drop(["RUType"], axis=1)

eval_data_1cell.to_csv("./prepared_data/Prediction_data_1cell.csv", index=False)

TWO CELL PREP

In [11]:
# condition = (eval_data_24cell['CellName'] != 'Cell2') & (eval_data_24cell['CellName'] != 'Cell3')
# eval_data_24cell = eval_data_24cell[condition]

joined_df_2 = eval_data_2cell.pivot_table(index=['Time', 'BS'], columns='CellName', values=['load', 'ESMode1', 'ESMode2', 'ESMode3',
       'ESMode4', 'ESMode5', 'ESMode6', 'Frequency', 'Bandwidth', 'TXpower'], aggfunc='sum', fill_value=999)

joined_df_2.columns = ['{}_{}'.format(col[1], col[0]) for col in joined_df_2.columns]
joined_df_2 = joined_df_2.sort_index(axis=1)
joined_df_2 = joined_df_2.reset_index()
joined_df_2 = joined_df_2.merge(eval_data_2cell[['Time', 'BS', 'RUType', 'Mode', 'Antennas']], on=['Time', 'BS'], how='inner')
joined_df_2 = joined_df_2.drop_duplicates()
joined_df_2 = joined_df_2.sort_values(by=['BS','Time'])
# joined_df_2.to_csv('./a.csv')

In [12]:
joined_df_2['Time'] = pd.to_datetime(joined_df_2['Time'], format='%m/%d/%Y %H:%M')

joined_df_2['Year'] = joined_df_2['Time'].dt.year
joined_df_2['Day'] = joined_df_2['Time'].dt.day
joined_df_2['Hour'] = joined_df_2['Time'].dt.hour
joined_df_2['Month'] = joined_df_2['Time'].dt.month

In [13]:
with open('./pickle/past_energy_cell2_dict.pkl', 'rb') as file:
    past_energy_cell2_dict = pickle.load(file)
joined_df_2['Energy_1'] = np.zeros(len(joined_df_2))
for row in joined_df_2.itertuples():
    # print((row.BS,row.Time))
    joined_df_2.at[row.Index, 'Energy_1'] = past_energy_cell2_dict[(row.BS,row.Time)]

In [14]:
with open('./pickle/big_dict.pkl', 'rb') as file:
    big_dict = pickle.load(file)

# uzupelnianie joined_df_2
for index, row in joined_df_2.iterrows():
    if row.Cell1_Bandwidth == 999:
        tmp_list = big_dict[(row.BS, row.Hour)]
        joined_df_2.at[index, 'Cell1_Bandwidth'] = tmp_list[0]
        joined_df_2.at[index, 'Cell1_ESMode1'] = tmp_list[1]
        joined_df_2.at[index, 'Cell1_ESMode2'] = tmp_list[2]
        joined_df_2.at[index, 'Cell1_ESMode3'] = tmp_list[3]
        joined_df_2.at[index, 'Cell1_ESMode4'] = tmp_list[4]
        joined_df_2.at[index, 'Cell1_ESMode5'] = tmp_list[5]
        joined_df_2.at[index, 'Cell1_ESMode6'] = tmp_list[6]
        joined_df_2.at[index, 'Cell1_Frequency'] = tmp_list[7]
        joined_df_2.at[index, 'Cell1_TXpower'] = tmp_list[8]
        joined_df_2.at[index, 'Cell1_load'] = tmp_list[9]
# joined_df_2.to_csv('./joined_df_2.csv', index=False)
# print(joined_df_2.value_counts('BS'))

In [15]:
label_encoders = {}
object_cols = ['RUType', 'Mode'] 

for col in object_cols:
    le = LabelEncoder()
    joined_df_2[col] = le.fit_transform(joined_df_2[col])
    label_encoders[col] = le
joined_df_2['RUType'].value_counts()

RUType
0    4969
3     239
1      48
2      35
4      31
5       4
Name: count, dtype: int64

In [16]:
Encoded_rutype = torch.nn.functional.one_hot(torch.tensor(joined_df_2['RUType'].values).long(), 6)
Encoded_rutype = pd.DataFrame(Encoded_rutype, columns=[f"RUType{i}" for i in range(6)])
Encoded_rutype.value_counts()

RUType0  RUType1  RUType2  RUType3  RUType4  RUType5
1        0        0        0        0        0          4969
0        0        0        1        0        0           239
         1        0        0        0        0            48
         0        1        0        0        0            35
                  0        0        1        0            31
                                    0        1             4
Name: count, dtype: int64

In [17]:
joined_df_2_white_man = joined_df_2.to_numpy()
Encoded_rutype_white_man = Encoded_rutype.to_numpy()
chuj = joined_df_2.columns
joined_df_2 = np.concatenate((joined_df_2_white_man,Encoded_rutype_white_man), axis=1)
joined_df_2 = pd.DataFrame(joined_df_2, columns = chuj.append(Encoded_rutype.columns))
# joined_df_2 = pd.concat([joined_df_2, Encoded_rutype], axis=1, ignore_index=)

# joined_df_2 = joined_df_2.drop(["BS", "Year", "Month", "RUType", "Time"], axis=1)
joined_df_2 = joined_df_2.drop(["Year", "Month", "RUType"], axis=1)
joined_df_2.to_csv("./prepared_data/Prediction_data_2cells.csv", index=False)
print(joined_df_2.columns)

Index(['Time', 'BS', 'Cell0_Bandwidth', 'Cell0_ESMode1', 'Cell0_ESMode2',
       'Cell0_ESMode3', 'Cell0_ESMode4', 'Cell0_ESMode5', 'Cell0_ESMode6',
       'Cell0_Frequency', 'Cell0_TXpower', 'Cell0_load', 'Cell1_Bandwidth',
       'Cell1_ESMode1', 'Cell1_ESMode2', 'Cell1_ESMode3', 'Cell1_ESMode4',
       'Cell1_ESMode5', 'Cell1_ESMode6', 'Cell1_Frequency', 'Cell1_TXpower',
       'Cell1_load', 'Mode', 'Antennas', 'Day', 'Hour', 'Energy_1', 'RUType0',
       'RUType1', 'RUType2', 'RUType3', 'RUType4', 'RUType5'],
      dtype='object')


FOUR CELL PREP

In [18]:
joined_df_4 = eval_data_4cell.pivot_table(index=['Time', 'BS'], columns='CellName', values=['load', 'ESMode1', 'ESMode2', 'ESMode3',
       'ESMode4', 'ESMode5', 'ESMode6', 'Frequency', 'Bandwidth', 'TXpower'], aggfunc='sum', fill_value=999)

joined_df_4.columns = ['{}_{}'.format(col[1], col[0]) for col in joined_df_4.columns]
joined_df_4 = joined_df_4.sort_index(axis=1)
joined_df_4 = joined_df_4.reset_index()
joined_df_4 = joined_df_4.merge(eval_data_4cell[['Time', 'BS', 'RUType', 'Mode', 'Antennas']], on=['Time', 'BS'], how='inner')
joined_df_4 = joined_df_4.drop_duplicates()
joined_df_4 = joined_df_4.sort_values(by=['BS','Time'])
# joined_df_4.to_csv('./a.csv')

In [19]:
joined_df_4['Time'] = pd.to_datetime(joined_df_4['Time'], format='%m/%d/%Y %H:%M')

joined_df_4['Year'] = joined_df_4['Time'].dt.year
joined_df_4['Day'] = joined_df_4['Time'].dt.day
joined_df_4['Hour'] = joined_df_4['Time'].dt.hour
joined_df_4['Month'] = joined_df_4['Time'].dt.month

In [20]:
with open('./pickle/past_energy_cell4_dict.pkl', 'rb') as file:
    past_energy_cell4_dict = pickle.load(file)
joined_df_4['Energy_1'] = np.zeros(len(joined_df_4))
for row in joined_df_4.itertuples():
    # print((row.BS,row.Time))
    joined_df_4.at[row.Index, 'Energy_1'] = past_energy_cell4_dict[(row.BS,row.Time)]

In [21]:
with open('./pickle/sec_third_cell_dict.pkl', 'rb') as file:
    sec_third_cell_dict = pickle.load(file)

for index, row in joined_df_4.iterrows():
    if row.Cell2_Bandwidth == 999:
        tmp_list = sec_third_cell_dict[(row.BS, row.Hour)]
        # print(tmp_list)
        joined_df_4.at[index, 'Cell2_Bandwidth'] = tmp_list[0]
        joined_df_4.at[index, 'Cell2_ESMode1'] = tmp_list[1]
        joined_df_4.at[index, 'Cell2_ESMode2'] = tmp_list[2]
        joined_df_4.at[index, 'Cell2_ESMode3'] = tmp_list[3]
        joined_df_4.at[index, 'Cell2_ESMode4'] = tmp_list[4]
        joined_df_4.at[index, 'Cell2_ESMode5'] = tmp_list[5]
        joined_df_4.at[index, 'Cell2_ESMode6'] = tmp_list[6]
        joined_df_4.at[index, 'Cell2_Frequency'] = tmp_list[7]
        joined_df_4.at[index, 'Cell2_TXpower'] = tmp_list[8]
        joined_df_4.at[index, 'Cell2_load'] = tmp_list[9]
        joined_df_4.at[index, 'Cell3_Bandwidth'] = tmp_list[10]
        joined_df_4.at[index, 'Cell3_ESMode1'] = tmp_list[11]
        joined_df_4.at[index, 'Cell3_ESMode2'] = tmp_list[12]
        joined_df_4.at[index, 'Cell3_ESMode3'] = tmp_list[13]
        joined_df_4.at[index, 'Cell3_ESMode4'] = tmp_list[14]
        joined_df_4.at[index, 'Cell3_ESMode5'] = tmp_list[15]
        joined_df_4.at[index, 'Cell3_ESMode6'] = tmp_list[16]
        joined_df_4.at[index, 'Cell3_Frequency'] = tmp_list[17]
        joined_df_4.at[index, 'Cell3_TXpower'] = tmp_list[18]
        joined_df_4.at[index, 'Cell3_load'] = tmp_list[19]
joined_df_4.to_csv('./joined_df_4.csv', index=False)

In [25]:
label_encoders = {}
object_cols = ['RUType', 'Mode'] 

for col in object_cols:
    le = LabelEncoder()
    joined_df_4[col] = le.fit_transform(joined_df_4[col])
    label_encoders[col] = le
joined_df_4['RUType'].value_counts()

RUType
0    65
Name: count, dtype: int64

In [26]:
# joined_df_4_white_man = joined_df_4.to_numpy()
# Encoded_rutype_white_man = Encoded_rutype.to_numpy()
# chuj = joined_df_4.columns
# joined_df_4 = np.concatenate((joined_df_4_white_man,Encoded_rutype_white_man), axis=1)
# joined_df_4 = pd.DataFrame(joined_df_4, columns = chuj.append(Encoded_rutype.columns))

# joined_df_4 = joined_df_4.drop(["BS", "Year", "Month", "RUType", "Time"], axis=1)
joined_df_4 = joined_df_4.drop(["Year", "Month", "RUType"], axis=1)
joined_df_4.to_csv("./prepared_data/Prediction_data_4cells.csv", index=False)
print(joined_df_4.columns)

Index(['Time', 'BS', 'Cell0_Bandwidth', 'Cell0_ESMode1', 'Cell0_ESMode2',
       'Cell0_ESMode3', 'Cell0_ESMode4', 'Cell0_ESMode5', 'Cell0_ESMode6',
       'Cell0_Frequency', 'Cell0_TXpower', 'Cell0_load', 'Cell1_Bandwidth',
       'Cell1_ESMode1', 'Cell1_ESMode2', 'Cell1_ESMode3', 'Cell1_ESMode4',
       'Cell1_ESMode5', 'Cell1_ESMode6', 'Cell1_Frequency', 'Cell1_TXpower',
       'Cell1_load', 'Cell2_Bandwidth', 'Cell2_ESMode1', 'Cell2_ESMode2',
       'Cell2_ESMode3', 'Cell2_ESMode4', 'Cell2_ESMode5', 'Cell2_ESMode6',
       'Cell2_Frequency', 'Cell2_TXpower', 'Cell2_load', 'Cell3_Bandwidth',
       'Cell3_ESMode1', 'Cell3_ESMode2', 'Cell3_ESMode3', 'Cell3_ESMode4',
       'Cell3_ESMode5', 'Cell3_ESMode6', 'Cell3_Frequency', 'Cell3_TXpower',
       'Cell3_load', 'Mode', 'Antennas', 'Day', 'Hour', 'Energy_1'],
      dtype='object')
